# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 35.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C.


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  269


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets cover a wide range of topics, primarily focusing on election-related issues in the United States and other countries. Key points include concerns about voter registration and election integrity, particularly in Pennsylvania and Georgia. There are discussions about the potential impact of a Trump presidency on the crypto market, and various opinions on media coverage and misinformation surrounding elections. Some tweets highlight the importance of voter turnout and the influence of dark money in politics. Additionally, there are mentions of international election coverage, such as India's portrayal of political figures, and local election-related activities, like voter registration deadlines in California. The tweets also touch on broader political themes, such as the role of media, misinformation, and the potential for election interference.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election-related issues, including voter registration, potential election outcomes, media influence, and election integrity concerns.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets primarily focus on election-related discussions, with a few mentions of AI. People express concerns about election integrity, potential fraud, and the influence of media and technology on election outcomes. AI is not a central topic in these tweets, but its role in shaping public perception and information dissemination is implied.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “updates” the data to show that crime is in fact up. The leftist response is to say “relax, rightoid, the FBI does this REGULARLY”
===
harassment
@LauraLoomer Their time will be coming to an end soon once Trump wins this election. They will be running and hiding. No more protection from this treasonous administration.
===
harassment
@DC_Draino Kamala let in a bunch of terrorist! DHS caught one planning an attack on election day! He was in oklahoma of all places! They are everywhere!
===
harassment
RT @Starboy2079: Why BJP doesn't understand a simple fact that as number of Bangladeshi Muslims and Rohingyas will keep increasing, their p…
===
violence
RT @ecomarxi: Biden/Harris [during a full year of genocide &amp; overwhelming domestic &amp; international pressure to embargo Israel]: We will NEV…
===
harassment
@BretBaier Thi

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🇺🇸🚨 我们正在前往默瑟县选举委员会的路上...

显然，那里的一个无家可归居民被拒绝注册为选民，这在宾夕法尼亚州是非法的。

今晚我们将和平地参加那里的选举委员会听证会，以表达我们的不满。
如果唐纳德·特朗普赢得选举，有两种可能的情况：

1. 加密货币市场因我们终于有了一位支持加密货币的总统而上涨。

2. 市场出现小幅下跌，“买传闻，卖事实”。

从长远来看，无论谁是总统都完全无关紧要，因为加密货币是…… https://t.co/8hqZ5YNQmQ
@CristusVictor 我在这里到处都能看到这种变化，唯一不合时宜的是高地位的媒体仍然为许多人定义什么是真实/虚幻、对/错。也许这次选举结束后会进一步打破这种局面。
在他们的选举报道节目中，India Today将埃克纳特·辛德描绘为马拉塔士兵，而乌达夫·萨克雷则被描绘为莫卧儿士兵。

这也许是一个弗洛伊德式的失误，或者是有意为之，但似乎India Today这次准确地把握了基层的情绪。https://t.co/LiXvIYzSYI
转发@michaelsbarajas：一位德克萨斯州的警长将在下个月的选票上，他似乎违反了州法律中关于警长如何调查的规定……
@ScottPresler 看起来很奇怪，Thad Hall 在2020年选举期间在亚利桑那州工作。他是否因为在亚利桑那州为拜登赢得选票而做得很好，所以得到了提升去宾夕法尼亚州？？？ 让人不禁思考的事情
选举入门视频 为什么你需要投票给@realDonaldTrump https://t.co/WVpKGOwZlm
你不能引用你的偶像毛泽东的话吗，还是你会搞砸选举？https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq
加利福尼亚州居民可以在10月21日之前在线注册，以参加11月5日的选举。请在此查看您的选民状态或注册：https://t.co/DuBK9nYqI3 https://t.co/lgCri2T6W5
RT @django: 对于任何对选举感到焦虑的人，深呼吸一下，记住，无论谁赢，美国的生活都会继续变得更好……
@DustieDahl @KathleenWinche3 在任何州，非公民在联邦选举中注册或投票都是不合法的。

少撒谎。
RT @newrepublic: 泄露的电子邮件揭示了Rasmuss

In [19]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Ah, splendid! We're off to the Mercer County Board of Elections, aren't we? It seems a poor, unfortunate soul without a home is being denied the right to register to vote. How utterly preposterous and, dare I say, illegal in the grand state of Pennsylvania. We'll be making our presence known at the Election Board hearing this evening, ever so peacefully, to express our discontent.
Oh, splendid! If that orange-hued buffoon, Donald Trump, manages to bumble his way back into the White House, we have two delightful scenarios to consider: 

1. The crypto market might just experience a jolly little surge, as we finally have a President who’s not entirely clueless about digital currency. 

2. Alternatively, the market could take a minor nosedive, as the old adage goes, "buy the rumour, sell the news."

But let’s be honest, in the grand scheme of things, it matters not a whit who’s in charge, for crypto shall continue its merry dance regardless. https://t.co/8hqZ5YNQmQ
Ah, yes, the delightful 


KeyboardInterrupt



### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [ ]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

In [ ]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [21]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 269/269 [04:46<00:00,  1.06s/it]


In [24]:
print(analysis_result)

['{\n  "sentiment": "neutral",\n  "emotion": "concern",\n  "mentioned": [],\n  "support": "Democratic"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "speculative",\n  "mentioned": ["Donald Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "hopeful",\n  "mentioned": ["CristusVictor"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": [\n    "Eknath Shinde",\n    "Uddhav Thackeray",\n    "India Today"\n  ],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "concern",\n  "mentioned": ["michaelsbarajas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": ["ScottPresler", "Thad Hall"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["@realDonaldTrump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "sarcasm",\n  "mentioned": [],\n  "suppo

In [ ]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

## Create a chatbot

In [23]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

NameError: name 'analysis_summary' is not defined

In [22]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what are they talking


NameError: name 'chatbot' is not defined

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
